# Development Notes

## Bitmath

There's a handy pypi/github package called `bitmath` that handles converting bytecounts into human readable strings - it also provides math operations between bytecount objects. Use this, if this functionality becomes needed.

## Children really do look like their parents
__note:__ `rclone` uses some interesting variants of the `ls` concept
- ls
- lsl
- lsd
- lsjson

Each one produces a slight different result depending on whether the items living at the requested path are "objects" or "containers". In cases where the parent container and the child object have the same name, the results can appear ambiguous.

For instance, the lsjson command will return a JSON list of one object if the path is a directory with only one child, and will also return a JSON list of one object if the path is a file -- the dirent is the file.

So for a two paths which point to the same level of hierarchy, you may receive data regarding objects from to different levels of hierarchy -- and it's up to you to distinguish which level.

The unix 'ls' command behaves in similar ways mind you -- not exactly the same, but can't really disambiguate alone.

#### The "is it cake?" test.

That's why we have 'stat' and 'file' commands. And pyfilesystem has 'getinfo()' and 'isdir()'.

So you want to get info for an item at the end of the path?

Just take a step back. You can get the JSON list of items belonging to the parent. Then pick that item's info out of the list. Then check the "IsDir" property.

So as a matter of principal, only run the `ls` commands on paths you _know_ are directories.

That kind of pathwalk is piece of cake.

Unless you input a non-existant path where the last segment has the same name as its parent, then you're guaranteed a good time.

#### Nailing the cake

God. We really have to do a path walk from root to avoid that one.

Or just run two `lsjson` operations. One on the parent, one on the presumed child. The item info for the child segment, extracted from the `lsjson` operation on the parent segment, is valid IF the `lsjson` operation on the child segment does not return "directory not found".

Makes perfect sense.

## The Root of the Trouble

Even though we can get info on the objects and containers within a remote. We can't get info on the "root" of the remote. That is '/'.

Because that evaluates to the remote itself. And you can't get the same JSON info on the remote as you can on its objects.

So we have to make up something.

### Is there ever a chance that a remote is not listable? Like the remote is just a file?
